In [78]:
import os
import sys
import pandas as pd
import numpy as np
from ast import literal_eval

os.getcwd()

'/Users/chris/GitHub/mathai/src'

## problems and problem set dataframes, refactoring

In [105]:
problems_df = pd.read_csv('../db/problems_df.csv', index_col='problem_ID')
cols_into_lists = {'problem_IDs':literal_eval, 'head':literal_eval, 'body':literal_eval, 'problems_list':literal_eval}
problem_sets_df = pd.read_csv('../db/problem_sets_df.csv', index_col='problem_set_ID', converters=cols_into_lists)

In [108]:
# Problems DF
display(problems_df.head(2))
print('problem count: ', len(problems_df))

question_lengths = problems_df.question.apply(len)
print('\nquestion text length min, mean, max:\n', question_lengths.min(), 
    round(question_lengths.mean()), question_lengths.max(), '\n')

#for col in problems_df.columns:
#    print(col, type(problems_df.loc[0,col]))

,problem_set_ID,question
problem_ID,,
0,0,Points that are all located on the same line a...
1,0,Use symbols to write the name of each geometri...


problem count:  1855

question text length min, mean, max:
 2 506 3217 

problem_set_ID <class 'numpy.int64'>
question <class 'str'>


In [137]:

# Problem Sets DF
#for col in problem_sets_df.columns:
#    print(col, type(problem_sets_df.loc[0,col]))
display(problem_sets_df[146:148])
print('pset count: ', len(problem_sets_df))

print('Average length of problems_list (num lines): \n', round(problem_sets_df.problems_list.str.len().mean()))

,filename,head,body,problems_list,problem_count,problem_IDs
problem_set_ID,,,,,,
146,/Users/chris/GitHub/course-files/Geometry/15-T...,"[\begin{document}\n, \subsubsection*{Transvers...","[ \item Corresponding angles\n, \item Alter...","[Corresponding angles\n, Alternate interior an...",6,"[1825, 1826, 1827, 1828, 1829, 1830]"
147,/Users/chris/GitHub/course-files/Geometry/15-T...,"[\begin{document}\n, \subsubsection*{Trig}\n]","[ \item Slope as $\tan \theta$\n, \item...","[Slope as $\tan \theta$\n, calculate sine, cos...",9,"[1831, 1832, 1833, 1834, 1835, 1836, 1837, 183..."


pset count:  149
Average length of problems_list (num lines): 
 12


In [154]:
n = 148
pset_list = problem_sets_df.loc[n, 'problems_list']
print(pset_list, type(pset_list), '\n', pset_list[0])
print(problem_sets_df.loc[147,'problem_IDs'][0])


['A(n) $\\rule{4cm}{0.15mm}$ is a straight continuous arrangement of an infinite number of points.\n', 'A(n) $\\rule{4cm}{0.15mm}$ is a portion of a line that begins with a single point and extends infinitely in one direction.\n', 'Points that are all located on the same line are $\\rule{4cm}{0.15mm}$.\n', 'The points where a line segment begins and ends are the $\\rule{4cm}{0.15mm}$.\n', 'Two or more line segments of equal measure are $\\rule{4cm}{0.15mm}$.\n', 'A(n) $\\rule{4cm}{0.15mm}$ is a portion of a line that includes two points and all of the collinear points between the two points.\n', 'A flat surface is a(n) $\\rule{4cm}{0.15mm}$.\n\n', 'Use each term according to its geometric meaning: ``sketch", ``draw", ``construct".\n', '$\\rule{4cm}{0.15mm}$ is to make a freehand diagram showing important features.\n', '$\\rule{4cm}{0.15mm}$ is to depict with accurate measures using ruler, protractor, and compass.\n', '$\\rule{4cm}{0.15mm}$ is a formal, logical process to create geometr

In [169]:
names = problem_sets_df.filename
names.apply(lambda x: x.split('/')[-2:]).iloc[120:124]

problem_set_ID
120      [13-Review, 13-01-DN-similarity-ratios.tex]
121         [13-Review, 13-01DN-transformations.tex]
122    [13-Review, 13-02DN+HW-similarity-ratios.tex]
123            [13-Review, 13-03HW-construction.tex]
Name: filename, dtype: object

In [194]:
names2 = problem_sets_df.filename.str.split('/')
display(names2)
file_list = pd.DataFrame()
file_list['chapter'] = names2.str.get(-2)
file_list['fn'] = names2.str.get(-1)
file_list

problem_set_ID
0      [, Users, chris, GitHub, course-files, Geometr...
1      [, Users, chris, GitHub, course-files, Geometr...
2      [, Users, chris, GitHub, course-files, Geometr...
3      [, Users, chris, GitHub, course-files, Geometr...
4      [, Users, chris, GitHub, course-files, Geometr...
                             ...                        
144    [, Users, chris, GitHub, course-files, Geometr...
145    [, Users, chris, GitHub, course-files, Geometr...
146    [, Users, chris, GitHub, course-files, Geometr...
147    [, Users, chris, GitHub, course-files, Geometr...
148    [, Users, chris, GitHub, course-files, Geometr...
Name: filename, Length: 149, dtype: object

,chapter,fn
problem_set_ID,,
0,01-Intro,1-Exam-Intro.tex
1,01-Intro,1-HW-Intro.tex
2,01-Intro,1b-Exam-Angle-pairs.tex
3,01-Intro,1b2-HW-Intro.tex
4,01-Intro,Exam-1-Intro.tex
...,...,...
144,15-Trajectories,Segments.tex
145,15-Trajectories,Transformations.tex
146,15-Trajectories,Transversals.tex


In [201]:
print(problems_df.loc[problem_sets_df.loc[117,'problem_IDs']].iloc[0]['question'])


Given parallelogram $ABCD$ with $m\angle A=75^\circ$, $AB=8$, and $BC=12$. Find the value of each angle measure or side length.
     \begin{enumerate}
       \item $m\angle B=$\vspace{0.5cm}
       \item $m\angle C=$\vspace{0.5cm}
       \item $m\angle D=$\vspace{0.5cm}
       \item $CD=$ \vspace{0.5cm}
       \item $AD=$ \vspace{0.5cm}
     \end{enumerate}




In [211]:
display(problem_sets_df.loc[117, 'body'][:9])
print(problem_sets_df.loc[117, 'problems_list'][:1][0])

['\n',
 '   \\item Given parallelogram $ABCD$ with $m\\angle A=75^\\circ$, $AB=8$, and $BC=12$. Find the value of each angle measure or side length.\n',
 '     \\begin{enumerate}\n',
 '       \\item $m\\angle B=$\\vspace{0.5cm}\n',
 '       \\item $m\\angle C=$\\vspace{0.5cm}\n',
 '       \\item $m\\angle D=$\\vspace{0.5cm}\n',
 '       \\item $CD=$ \\vspace{0.5cm}\n',
 '       \\item $AD=$ \\vspace{0.5cm}\n',
 '     \\end{enumerate}\n']

Given parallelogram $ABCD$ with $m\angle A=75^\circ$, $AB=8$, and $BC=12$. Find the value of each angle measure or side length.
     \begin{enumerate}
       \item $m\angle B=$\vspace{0.5cm}
       \item $m\angle C=$\vspace{0.5cm}
       \item $m\angle D=$\vspace{0.5cm}
       \item $CD=$ \vspace{0.5cm}
       \item $AD=$ \vspace{0.5cm}
     \end{enumerate}




In [24]:
short_df = problems_df[problems_df.problem_set_ID==7].head(3)
prefix = '\nNew Problem\n'
problem_tex = short_df.question.str.cat(sep=prefix)
print(problem_tex)

Identify a linear pair of angles in the given diagram.
    \begin{center}
    \begin{tikzpicture}
      %\draw [->, thick] (0,0)--(5,5);
      \draw [<-, thick] (-2,0)--(3,0)--(2,3)--(0,0);
      \draw [fill] (-1.5,0) circle [radius=0.05] node[below]{$R$};
      \draw [fill] (0,0) circle [radius=0.05] node[below]{$S$};
      \draw [fill] (2,3) circle [radius=0.05] node[right]{$U$};
      \draw [fill] (3,0) circle [radius=0.05] node[right]{$T$};
    \end{tikzpicture}
    \end{center}


New Problem
Identify three rays in the given plane.\\[0.25in]
      \begin{tikzpicture}[scale=0.8]
        \draw [thick](0,0) node[above right]{$\ m$} --(6,0)--(8,4)--(2,4)--(0,0);
        \draw [<->, thick] (1,1)--(6,3);
        \draw [fill] (3.5,2) circle [radius=0.05] node[below]{$P \ $};
        \draw [fill] (5,2.6) circle [radius=0.05] node[below]{$Q$};
        \draw [<->, thick] (2,3.5)--(5.25,.25);
        \draw [fill] (5,0.5) circle [radius=0.05] node[left]{$R \ $};
      \end{tikzpicture}
      \